In [2]:
pip install mlxtend

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------------------------------- -------- 1.0/1.4 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 5.5 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.1 MB 6.3 MB/s eta 0:00:02
   --------- ------------------------------ 2.6/11.1 MB 6.3 MB/s eta 0:00:02
   -------------- ------------------------- 3.9/11.1 MB 6.3 MB/s eta 0:00:02
   ------------------ --------------------- 5.2/11.1 MB 6.3 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/11.1 MB 6.3 MB/s eta 0:00:01
   ---------------------------- ----------- 7.9/11.1 MB 6.3 MB/s eta 0:00:01
   -------------------------------- ------- 8.9/11.1 MB 6.3 MB/s eta 0:00:01
   ------------------------------------ --- 10.2/11.1 M


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from mlxtend.frequent_patterns import fpgrowth,apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [5]:
data = pd.read_csv(r"D:\Personal Projects\Market Basket Analysis\groceries - groceries.csv")

In [6]:
data.head()

,Item(s),Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,...,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32
0,4,citrus fruit,semi-finished bread,margarine,ready soups,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,tropical fruit,yogurt,coffee,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,whole milk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,pip fruit,yogurt,cream cheese,meat spreads,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,other vegetables,whole milk,condensed milk,long life bakery product,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Reformatting the data
Now to compute any association rule or look at the itemsets and their supports we need to reformat the data.
So, let's bring the data into its right format.
We will use TransactionEncoder from mlxtrend for the formatting.

In [7]:
# Iterate using iloc[]

transactions = []
for i in range(len(data)):
    values = data.iloc[i,1:].values
#     print([x for x in values if x is not np.nan])
    transactions.append([x for x in values if x is not np.nan])

In [8]:
# Preprocessing using TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

In [9]:
df

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9830,False,False,False,False,False,False,False,False,False,True,...,False,False,False,True,False,False,False,True,False,False
9831,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9832,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
9833,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Exploratory Data Analysis
Now the data is ready, lets compute the support of each items and have a look at some most popular items and their transaction.
Also will see some of the low selling items as well.

In [10]:
# Computing support for individual items
item_support = df.sum() / len(df)

In [11]:
## Lets look at top 25 most selling items

item_support.sort_values(ascending=False).head(25)

whole milk               0.255516
other vegetables         0.193493
rolls/buns               0.183935
soda                     0.174377
yogurt                   0.139502
bottled water            0.110524
root vegetables          0.108998
tropical fruit           0.104931
shopping bags            0.098526
sausage                  0.093950
pastry                   0.088968
citrus fruit             0.082766
bottled beer             0.080529
newspapers               0.079817
canned beer              0.077682
pip fruit                0.075648
fruit/vegetable juice    0.072293
whipped/sour cream       0.071683
brown bread              0.064870
domestic eggs            0.063447
frankfurter              0.058973
margarine                0.058566
coffee                   0.058058
pork                     0.057651
butter                   0.055414
dtype: float64

So, items like whole milk , vegetables are at top spots. Most of the top sellings are milk and dairy products, vegetables and fruits and beverages, which is expected in most of the retail stores.

Now lets also look at some low selling items in the same store.

In [12]:
item_support.sort_values(ascending=False).tail(25)

cocoa drinks              0.002237
organic sausage           0.002237
prosecco                  0.002034
flower soil/fertilizer    0.001932
ready soups               0.001830
specialty vegetables      0.001729
organic products          0.001627
honey                     0.001525
decalcifier               0.001525
cream                     0.001322
frozen fruits             0.001220
hair spray                0.001118
rubbing alcohol           0.001017
liqueur                   0.000915
whisky                    0.000813
make up remover           0.000813
salad dressing            0.000813
toilet cleaner            0.000712
frozen chicken            0.000610
baby cosmetics            0.000610
bags                      0.000407
kitchen utensil           0.000407
preservation products     0.000203
baby food                 0.000102
sound storage medium      0.000102
dtype: float64

We can see some frozen products , alcoholic beverages and homecares etc.

So we can assume the store is popular in fresh groceires rather than packaged items.

### Creating Frequent itemsets
Now lets create itemsets pairs from the transactions data. And then look at some of the most sold itemssets also low sold ones. We will use Fpgrowth algorithm as this is high efficient and time saving.

fpgrowth algorithms expects to set a min_support values for filtering out insignificant items.

Choosing this threshold is very critical since a small value might include some less important itemsets into the results and high value might results in only few itemsets.

In [13]:
# Apply FP-Growth algorithm
frequent_itemsets = fpgrowth(df, min_support=0.01, use_colnames=True)

In [14]:
frequent_itemsets

,support,itemsets
0,0.082766,(citrus fruit)
1,0.058566,(margarine)
2,0.017692,(semi-finished bread)
3,0.139502,(yogurt)
4,0.104931,(tropical fruit)
...,...,...
328,0.010168,"(frozen vegetables, rolls/buns)"
329,0.012405,"(frozen vegetables, yogurt)"
330,0.014235,"(onions, other vegetables)"
331,0.012100,"(onions, whole milk)"


In [15]:
## Lets filter for itemsets with more than one single item 

filtered_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: len(x) > 1)]

Lets look at some top itemsets and their support values.

In [16]:
print(filtered_itemsets.sort_values(by='support',ascending=False).head(25)[['itemsets', 'support']])

                                   itemsets   support
146          (whole milk, other vegetables)  0.074835
157                (rolls/buns, whole milk)  0.056634
107                    (yogurt, whole milk)  0.056024
240           (root vegetables, whole milk)  0.048907
238     (root vegetables, other vegetables)  0.047382
110              (yogurt, other vegetables)  0.043416
156          (rolls/buns, other vegetables)  0.042603
118            (tropical fruit, whole milk)  0.042298
191                      (soda, whole milk)  0.040061
190                      (rolls/buns, soda)  0.038332
116      (tropical fruit, other vegetables)  0.035892
165             (bottled water, whole milk)  0.034367
109                    (rolls/buns, yogurt)  0.034367
227                    (whole milk, pastry)  0.033249
192                (soda, other vegetables)  0.032740
291        (whipped/sour cream, whole milk)  0.032232
256                   (rolls/buns, sausage)  0.030605
88               (citrus fru

Top itemsets all contains combinations of dairy products and vegetables which supports our assumptions of selling fresh groceries.

Also the first itemset whole-milk and other vegetables also makes sense, since milk can be a high sold item and people could be buying some vegetables as well (since other vegetables includes set of many vegetables).

In [17]:
## bottom 25 itemsets

print(filtered_itemsets.sort_values(by='support',ascending=False).tail(25)[['itemsets', 'support']])
            

                                              itemsets   support
111                         (yogurt, soda, whole milk)  0.010473
222              (citrus fruit, fruit/vegetable juice)  0.010371
237                    (butter milk, other vegetables)  0.010371
99   (citrus fruit, root vegetables, other vegetables)  0.010371
319                      (citrus fruit, domestic eggs)  0.010371
104                         (bottled water, margarine)  0.010269
97                  (citrus fruit, whole milk, yogurt)  0.010269
176                                (soda, white bread)  0.010269
179                             (tropical fruit, curd)  0.010269
309                        (berries, other vegetables)  0.010269
307               (pork, whole milk, other vegetables)  0.010168
328                    (frozen vegetables, rolls/buns)  0.010168
198                     (root vegetables, frankfurter)  0.010168
154                       (butter, whipped/sour cream)  0.010168
106                      

### Association Rules:
Now that you have the frequent itemsets, we can generate association rules based on support, confidence, and lift metrics. As you mentioned, these metrics are fundamental for understanding relationships between products.

Here’s a breakdown of the metrics you’ll use:

Support (A ⇒ B): The proportion of transactions that include both item A and item B.

Confidence (A ⇒ B): The likelihood that item B will be purchased when item A is purchased.

Lift (A ⇒ B): Measures the strength of the rule by comparing the observed support to the expected support if items A and B were independent.

In [20]:
## Creating Association rule

rules = association_rules(frequent_itemsets,metric='confidence',min_threshold=0.5)

In [21]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,"(citrus fruit, root vegetables)",(other vegetables),0.017692,0.193493,0.010371,0.586207,3.029608,1.0,0.006948,1.949059,0.681990,0.051646,0.486932,0.319903
1,"(yogurt, other vegetables)",(whole milk),0.043416,0.255516,0.022267,0.512881,2.007235,1.0,0.011174,1.528340,0.524577,0.080485,0.345695,0.300014
2,"(tropical fruit, yogurt)",(whole milk),0.029283,0.255516,0.015150,0.517361,2.024770,1.0,0.007668,1.542528,0.521384,0.056184,0.351714,0.288326
3,"(tropical fruit, root vegetables)",(other vegetables),0.021047,0.193493,0.012303,0.584541,3.020999,1.0,0.008231,1.941244,0.683367,0.060835,0.484867,0.324062
4,"(tropical fruit, root vegetables)",(whole milk),0.021047,0.255516,0.011998,0.570048,2.230969,1.0,0.006620,1.731553,0.563627,0.045350,0.422484,0.308502
5,"(pip fruit, other vegetables)",(whole milk),0.026131,0.255516,0.013523,0.517510,2.025351,1.0,0.006846,1.543003,0.519843,0.050436,0.351913,0.285217
6,"(butter, other vegetables)",(whole milk),0.020031,0.255516,0.011490,0.573604,2.244885,1.0,0.006371,1.745992,0.565878,0.043512,0.427260,0.309285
7,"(curd, yogurt)",(whole milk),0.017285,0.255516,0.010066,0.582353,2.279125,1.0,0.005649,1.782567,0.571107,0.038313,0.439011,0.310874
8,"(rolls/buns, root vegetables)",(other vegetables),0.024301,0.193493,0.012201,0.502092,2.594890,1.0,0.007499,1.619792,0.629935,0.059347,0.382637,0.282575
9,"(rolls/buns, root vegetables)",(whole milk),0.024301,0.255516,0.012710,0.523013,2.046888,1.0,0.006500,1.560804,0.524192,0.047583,0.359305,0.286377


### Interpretation
### 1. Support

Definition: The proportion of transactions in the dataset that contain the itemset (both antecedent and consequent)
Gives us the idea of popularity of the itemsets.

In [23]:
rules.sort_values(by='support',ascending=False).head(10)[['antecedents','consequents','antecedent support','consequent support','support']]

,antecedents,consequents,antecedent support,consequent support,support
1,"(yogurt, other vegetables)",(whole milk),0.043416,0.255516,0.022267
2,"(tropical fruit, yogurt)",(whole milk),0.029283,0.255516,0.015150
12,"(whipped/sour cream, other vegetables)",(whole milk),0.028876,0.255516,0.014642
10,"(root vegetables, yogurt)",(whole milk),0.025826,0.255516,0.014540
5,"(pip fruit, other vegetables)",(whole milk),0.026131,0.255516,0.013523
11,"(root vegetables, yogurt)",(other vegetables),0.025826,0.193493,0.012913
9,"(rolls/buns, root vegetables)",(whole milk),0.024301,0.255516,0.012710
14,"(domestic eggs, other vegetables)",(whole milk),0.022267,0.255516,0.012303
3,"(tropical fruit, root vegetables)",(other vegetables),0.021047,0.193493,0.012303
8,"(rolls/buns, root vegetables)",(other vegetables),0.024301,0.193493,0.012201


We can see whole milk and other vegetables are very common in most of the frequent purchases which have higher support values. Support value tells us these items are very common in the dataset.

But the support value does not give us anything about antecedents impacts on the consequents. Example, in the above examples by looking at support value we can not conclude the purchase of yogurt and other vegetables (1st example) results in purchase of whole milk. It only tells us these itemsets are very common in the dataset.

### 2. Confidence

Definition: The likelihood that the consequent is purchased when the antecedent is purchased.
Evaluating the strength of the antecedent to the consequent

In [24]:
rules.sort_values(by='support',ascending=False).head(10)[['antecedents','consequents','support','confidence']]

,antecedents,consequents,support,confidence
1,"(yogurt, other vegetables)",(whole milk),0.022267,0.512881
2,"(tropical fruit, yogurt)",(whole milk),0.015150,0.517361
12,"(whipped/sour cream, other vegetables)",(whole milk),0.014642,0.507042
10,"(root vegetables, yogurt)",(whole milk),0.014540,0.562992
5,"(pip fruit, other vegetables)",(whole milk),0.013523,0.517510
11,"(root vegetables, yogurt)",(other vegetables),0.012913,0.500000
9,"(rolls/buns, root vegetables)",(whole milk),0.012710,0.523013
14,"(domestic eggs, other vegetables)",(whole milk),0.012303,0.552511
3,"(tropical fruit, root vegetables)",(other vegetables),0.012303,0.584541
8,"(rolls/buns, root vegetables)",(other vegetables),0.012201,0.502092


For the same examples where we had seen high support values, we can look at their confidence values. This value gives tells us how likely a person will buy whole milk if he already have "yogur and other vegetables" in his cart.

So its 51% likely that they will add whole milk into their cart.

Now lets have a look at rules/itemsets whith higher confidence.

In [25]:
rules.sort_values(by='confidence',ascending=False).head(10)[['antecedents','consequents','support','confidence']]

,antecedents,consequents,support,confidence
0,"(citrus fruit, root vegetables)",(other vegetables),0.010371,0.586207
3,"(tropical fruit, root vegetables)",(other vegetables),0.012303,0.584541
7,"(curd, yogurt)",(whole milk),0.010066,0.582353
6,"(butter, other vegetables)",(whole milk),0.011490,0.573604
4,"(tropical fruit, root vegetables)",(whole milk),0.011998,0.570048
10,"(root vegetables, yogurt)",(whole milk),0.014540,0.562992
14,"(domestic eggs, other vegetables)",(whole milk),0.012303,0.552511
13,"(whipped/sour cream, yogurt)",(whole milk),0.010880,0.524510
9,"(rolls/buns, root vegetables)",(whole milk),0.012710,0.523013
5,"(pip fruit, other vegetables)",(whole milk),0.013523,0.517510


Here we see confidence of 0.58 which means people who are buying these antecedents are 58% likely to buy the consequents items!!!!

In case any case we can notice that whole milk and other vegetables are common consequents in most of the itemsets. Its like every trasactions which include all other items also includes whole milk.

Does that mean whole milk are purchased as a complimentary item all the item ??? Are there any significant transactions that has only whole milk independently ????

These questions are answered by the Lift metric...

### 3. Lift

Definition: A measure of how much more likely the consequent is purchased when the antecedent is present compared to the situation where they are independent.

In [27]:
rules.sort_values(by='support',ascending=False).head(10)[['antecedents','consequents','support','confidence','lift']]

,antecedents,consequents,support,confidence,lift
1,"(yogurt, other vegetables)",(whole milk),0.022267,0.512881,2.007235
2,"(tropical fruit, yogurt)",(whole milk),0.015150,0.517361,2.024770
12,"(whipped/sour cream, other vegetables)",(whole milk),0.014642,0.507042,1.984385
10,"(root vegetables, yogurt)",(whole milk),0.014540,0.562992,2.203354
5,"(pip fruit, other vegetables)",(whole milk),0.013523,0.517510,2.025351
11,"(root vegetables, yogurt)",(other vegetables),0.012913,0.500000,2.584078
9,"(rolls/buns, root vegetables)",(whole milk),0.012710,0.523013,2.046888
14,"(domestic eggs, other vegetables)",(whole milk),0.012303,0.552511,2.162336
3,"(tropical fruit, root vegetables)",(other vegetables),0.012303,0.584541,3.020999
8,"(rolls/buns, root vegetables)",(other vegetables),0.012201,0.502092,2.594890


So, in the above examples it says when a person have "yogurt and other vegetables" in their basket they are 100% likely to buy whole milk compared to when buying nothing in their basket (or buying whole milk independetly).

Lets filter out cases where we have good support values to ensure reliability

In [28]:
rules = rules[rules['support']>0.0013]

In [29]:
rules.sort_values(by='support',ascending=False).head(10)[['antecedents','consequents','support','confidence','lift']]

,antecedents,consequents,support,confidence,lift
1,"(yogurt, other vegetables)",(whole milk),0.022267,0.512881,2.007235
2,"(tropical fruit, yogurt)",(whole milk),0.015150,0.517361,2.024770
12,"(whipped/sour cream, other vegetables)",(whole milk),0.014642,0.507042,1.984385
10,"(root vegetables, yogurt)",(whole milk),0.014540,0.562992,2.203354
5,"(pip fruit, other vegetables)",(whole milk),0.013523,0.517510,2.025351
11,"(root vegetables, yogurt)",(other vegetables),0.012913,0.500000,2.584078
9,"(rolls/buns, root vegetables)",(whole milk),0.012710,0.523013,2.046888
14,"(domestic eggs, other vegetables)",(whole milk),0.012303,0.552511,2.162336
3,"(tropical fruit, root vegetables)",(other vegetables),0.012303,0.584541,3.020999
8,"(rolls/buns, root vegetables)",(other vegetables),0.012201,0.502092,2.594890


Now lets again look at the first example where whole milk have good association with yogurt, other vegetables purchase.

Now, lets see these items support values individually

In [30]:
print(f"Support of Yogurt : {item_support['yogurt']}")
print(f"Support of  other vegetables : {item_support['other vegetables']}")
print(f"Support of whole milk : {item_support['whole milk']}")

Support of Yogurt : 0.13950177935943062
Support of  other vegetables : 0.1934926283680732
Support of whole milk : 0.25551601423487547


As we can see support of Yogurt is 0.14 , other vegetables is 0.19 and whole milk is 0.25

Which means 14% transactions consiste of yogurt and so on..

And the support value of all these 3 items is 0.022. Means 2.2% of the transactions contains Yogurt, other vegetables and milk together.

So by looking these values we can see the individual support of these items are much higher than when they are together..

So we have a seperate metric to check this values as well, which says if the items are purchased more frequently togthere than purchasing idependetly.

### 4. Leverage
   
Definition: The difference between the observed frequency of the itemset and the expected frequency if the two items were independent.

In [31]:
rules.sort_values(by='support',ascending=False).head(10)[['antecedents','consequents','support','confidence','lift','leverage']]

,antecedents,consequents,support,confidence,lift,leverage
1,"(yogurt, other vegetables)",(whole milk),0.022267,0.512881,2.007235,0.011174
2,"(tropical fruit, yogurt)",(whole milk),0.015150,0.517361,2.024770,0.007668
12,"(whipped/sour cream, other vegetables)",(whole milk),0.014642,0.507042,1.984385,0.007263
10,"(root vegetables, yogurt)",(whole milk),0.014540,0.562992,2.203354,0.007941
5,"(pip fruit, other vegetables)",(whole milk),0.013523,0.517510,2.025351,0.006846
11,"(root vegetables, yogurt)",(other vegetables),0.012913,0.500000,2.584078,0.007916
9,"(rolls/buns, root vegetables)",(whole milk),0.012710,0.523013,2.046888,0.006500
14,"(domestic eggs, other vegetables)",(whole milk),0.012303,0.552511,2.162336,0.006613
3,"(tropical fruit, root vegetables)",(other vegetables),0.012303,0.584541,3.020999,0.008231
8,"(rolls/buns, root vegetables)",(other vegetables),0.012201,0.502092,2.594890,0.007499


For the first example we can say that leverage is 0.011174, which is a positive number. Which suggests the items yogurt, other vegetables and whole milk does have strong association and they do have a significant impact to the overall trasnactions when they are available together.

### 5. Conviction
Definition: Conviction is a metric used in association rule mining to evaluate the strength of a rule. It helps in understanding how much more likely the consequent (the item being predicted) is to occur when the antecedent (the item that is known) is present, compared to when it is absent.

In [32]:
rules.sort_values(by='support',ascending=False).head(10)[['antecedents','consequents','support','confidence','lift','leverage','conviction']]

,antecedents,consequents,support,confidence,lift,leverage,conviction
1,"(yogurt, other vegetables)",(whole milk),0.022267,0.512881,2.007235,0.011174,1.528340
2,"(tropical fruit, yogurt)",(whole milk),0.015150,0.517361,2.024770,0.007668,1.542528
12,"(whipped/sour cream, other vegetables)",(whole milk),0.014642,0.507042,1.984385,0.007263,1.510239
10,"(root vegetables, yogurt)",(whole milk),0.014540,0.562992,2.203354,0.007941,1.703594
5,"(pip fruit, other vegetables)",(whole milk),0.013523,0.517510,2.025351,0.006846,1.543003
11,"(root vegetables, yogurt)",(other vegetables),0.012913,0.500000,2.584078,0.007916,1.613015
9,"(rolls/buns, root vegetables)",(whole milk),0.012710,0.523013,2.046888,0.006500,1.560804
14,"(domestic eggs, other vegetables)",(whole milk),0.012303,0.552511,2.162336,0.006613,1.663694
3,"(tropical fruit, root vegetables)",(other vegetables),0.012303,0.584541,3.020999,0.008231,1.941244
8,"(rolls/buns, root vegetables)",(other vegetables),0.012201,0.502092,2.594890,0.007499,1.619792


Conviction gives us the strength of the association rule by checking likeliness of consequents purchase when there is no antecedents .

In the table we can see whole milk is a consequents for most of the itemsets. Which means people are buying whole milk when they already have these antecedents in their basekt.

But how strong these asscoiations are ?

So in the first example we can say people are 1.5 times more likely to buy whole milk when they have yogurt and other vegetables in their basket compared to when they dont have them in their basket.

### Conclusion
The association rules provides us in-depth view about the market-basket analysis result.
The variuos metrics of the output helps us use association rules in different use cases like recommendation systems, cross selling , promotions etc.
So these metrics can be used in combinations to come up with right itemsets as per the requirement.